In [45]:
import pandas as pd  
import numpy as np
import math as math
from nltk.corpus import stopwords 
from nltk import word_tokenize
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')
import operator
import collections
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = TfidfVectorizer()
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kingalandydy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
import os
import re
import spacy
from gensim.models.phrases import Phrases, Phraser
from time import time 
import multiprocessing
from gensim.models import Word2Vec
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import scale
import keras 
from keras.models import Sequential, Model 
from keras import layers
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Input, Embedding
from keras.layers.merge import Concatenate
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics import confusion_matrix

In [47]:
xls = pd.ExcelFile('./Orbis - Energy.xlsx')
df = pd.read_excel(xls, 'Results')
#remove ones without description
df = df[~ pd.isnull(df['Description and history'])]
#remove the history
df['description'] = df['Description and history'].str.split('history').str[0]
df["energy"] = 1
df = df[['description','energy']]
df.head()

,description,energy
4.0,The Company is a France-based electricity prod...,1
5.0,"The Company was created on November 27, 1962 I...",1
6.0,The Company is a distributor of natural gas. N...,1
7.0,The Company specializes in electricity and gas...,1
8.0,RWE is the electricity and gas companies. Thro...,1


In [48]:
xls = pd.ExcelFile('./Orbis - Non-Energy.xlsx')
df2 = pd.read_excel(xls, 'Results')
#remove ones without description
df2 = df2[~ pd.isnull(df2['Description and history'])]
#remove the history
df2['description'] = df2['Description and history'].str.split('history').str[0]
df2["energy"] = 0
df2 = df2[['description','energy']]
df2.head()

,description,energy
4.0,"AT&T Inc., incorporated October 5, 1983, is a ...",0
6.0,"The company is engaged in various sectors, inc...",0
8.0,"Alphabet Inc., incorporated on July 23, 2015, ...",0
10.0,"Verizon Communications Inc., incorporated on O...",0
14.0,The Company is a Japan-based telecommunication...,0


In [49]:
dfx = pd.concat([df,df2])
df = dfx
df.head()

,description,energy
4.0,The Company is a France-based electricity prod...,1
5.0,"The Company was created on November 27, 1962 I...",1
6.0,The Company is a distributor of natural gas. N...,1
7.0,The Company specializes in electricity and gas...,1
8.0,RWE is the electricity and gas companies. Thro...,1


In [50]:
descList = []
df['description'] = df['description'].astype(str)
df['description'] = df['description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['description'] = df['description'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))
for description in df['description'].iteritems():
    descList.append(nltk.word_tokenize(description[1].lower()))
df['tokenized'] = descList

In [51]:
df.head()

,description,energy,tokenized
4.0,"the compani france-bas electr producer, market...",1,"[the, compani, france-bas, electr, producer, ,..."
5.0,"the compani creat novemb 27, 1962 it oper seve...",1,"[the, compani, creat, novemb, 27, ,, 1962, it,..."
6.0,the compani distributor natur gas. net sale br...,1,"[the, compani, distributor, natur, gas, ., net..."
7.0,the compani special electr gas product relat s...,1,"[the, compani, special, electr, gas, product, ..."
8.0,rwe electr gas companies. through expertis oil...,1,"[rwe, electr, gas, companies, ., through, expe..."


In [52]:
import gensim
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [53]:
#I tried training my own model but it was not enough words
#WORD2VEC()
#cores = multiprocessing.cpu_count() # Count the number of cores in a computer, important for a parameter of the model
#w2v_model = Word2Vec(min_count=20,
#                     window=2,
#                     size=300,
#                     sample=6e-5, 
#                     alpha=0.03, 
#                     min_alpha=0.0007, 
#                     negative=20,
#                     workers=cores-1)

#BUILD_VOCAB()
#t = time()
#w2v_model.build_vocab(df_clean["tokens"], progress_per=1000)
#print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

#TRAIN()
#w2v_model.train(df_clean["tokens"], total_examples=w2v_model.corpus_count, epochs=10000, report_delay=1)
#print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

In [54]:
#Splitting data
#First defining the X (input), and the y (output)
y = df['energy'].values
X = np.array(df["tokenized"])

#And here is the train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, random_state = 0)

In [55]:
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x for x in X_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

vocab size : 10922


In [56]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += w2v_model[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [57]:
train_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, X_train)])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, X_test)])
test_vecs_w2v = scale(test_vecs_w2v)

print ('shape for training set : ',train_vecs_w2v.shape,
      '\nshape for test set : ', test_vecs_w2v.shape)

shape for training set :  (17743, 300) 
shape for test set :  (934, 300)


In [58]:
model = Sequential()

model.add(Dense(128, activation='relu', input_dim=300))
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 128)               38528     
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 129       
Total params: 38,657
Trainable params: 38,657
Non-trainable params: 0
_________________________________________________________________


In [59]:
history = model.fit(train_vecs_w2v, y_train, epochs=30, batch_size=40,
                   validation_data=(test_vecs_w2v,y_test))
loss, accuracy = model.evaluate(train_vecs_w2v, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(test_vecs_w2v, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Train on 17743 samples, validate on 934 samples
Epoch 1/30
17743/17743 [==============================] - 4s 245us/step - loss: 0.2918 - accuracy: 0.9013 - val_loss: 0.1112 - val_accuracy: 0.9679
Epoch 2/30
17743/17743 [==============================] - 3s 171us/step - loss: 0.1798 - accuracy: 0.9474 - val_loss: 0.1052 - val_accuracy: 0.9700
Epoch 3/30
17743/17743 [==============================] - 3s 174us/step - loss: 0.1524 - accuracy: 0.9531 - val_loss: 0.1026 - val_accuracy: 0.9711
Epoch 4/30
17743/17743 [==============================] - 3s 179us/step - loss: 0.1481 - accuracy: 0.9553 - val_loss: 0.1035 - val_accuracy: 0.9700
Epoch 5/30
17743/17743 [==============================] - 3s 178us/step - loss: 0.1412 - accuracy: 0.9565 - val_loss: 0.0987 - val_accuracy: 0.9690
Epoch 6/30
17743/17743 [==============================] - 3s 186us/step - loss: 0.1367 - accuracy: 0.9577 - val_loss: 0.0991 - val_accuracy: 0.9679
Epoch 7/30
17743/17743 [==============================] - 3s 194

In [67]:
all_words = [word for tokens in X for word in tokens]
all_sentence_lengths = [len(tokens) for tokens in X]
ALL_VOCAB = sorted(list(set(all_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_words), len(ALL_VOCAB)))
print("Max sentence length is %s" % max(all_sentence_lengths))


####################### CHANGE THE PARAMETERS HERE #####################################
EMBEDDING_DIM = 300 # how big is each word vector
MAX_VOCAB_SIZE = 18399# how many unique words to use (i.e num rows in embedding vector)
MAX_SEQUENCE_LENGTH = 200 # max number of words in a comment to use (was 53)

3741498 words total, with a vocabulary size of 130977
Max sentence length is 4338


In [68]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(df["description"].tolist())
training_sequences = tokenizer.texts_to_sequences(X_train.tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = w2v_model[word] if word in w2v_model else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)


######################## TRAIN AND TEST SET #################################
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_sequences = tokenizer.texts_to_sequences(X_test.tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 98348 unique tokens.
(98349, 300)


In [69]:
from keras.layers import concatenate
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, trainable=False, extra_conv=True):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=trainable)

    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    convs = []
    filter_sizes = [3,4,5]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=3)(l_conv)
        convs.append(l_pool)

    l_merge = concatenate([convs[0],convs[1],convs[2]],axis=1)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    conv = Conv1D(filters=128, kernel_size=3, activation='relu')(embedded_sequences)
    pool = MaxPooling1D(pool_size=3)(conv)

    if extra_conv==True:
        x = Dropout(0.6)(l_merge)  
    else:
        # Original Yoon Kim model
        x = Dropout(0.5)(pool)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    # Finally, we feed the output into a Sigmoid layer.
    # The reason why sigmoid is used is because we are trying to achieve a binary classification(1,0) 
    # for each of the 6 labels, and the sigmoid function will squash the output between the bounds of 0 and 1.
    preds = Dense(1,activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adadelta',
                  metrics=['acc'])
    model.summary()
    return model

In [70]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, False)


Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 300, 300)     29504700    input_7[0][0]                    
__________________________________________________________________________________________________
conv1d_25 (Conv1D)              (None, 298, 128)     115328      embedding_7[0][0]                
__________________________________________________________________________________________________
conv1d_26 (Conv1D)              (None, 297, 128)     153728      embedding_7[0][0]                
____________________________________________________________________________________________

In [71]:
history = model.fit(train_cnn_data, y_train, epochs=10, batch_size=50,
                   validation_data=(test_cnn_data,y_test))
loss, accuracy = model.evaluate(train_cnn_data, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(test_cnn_data, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Train on 17743 samples, validate on 934 samples
Epoch 1/10
17743/17743 [==============================] - 206s 12ms/step - loss: 0.2756 - acc: 0.8960 - val_loss: 0.1215 - val_acc: 0.9593
Epoch 2/10
17743/17743 [==============================] - 193s 11ms/step - loss: 0.1209 - acc: 0.9587 - val_loss: 0.1337 - val_acc: 0.9657
Epoch 3/10
17743/17743 [==============================] - 195s 11ms/step - loss: 0.1004 - acc: 0.9660 - val_loss: 0.1045 - val_acc: 0.9679
Epoch 4/10
17743/17743 [==============================] - 193s 11ms/step - loss: 0.0837 - acc: 0.9711 - val_loss: 0.0963 - val_acc: 0.9754
Epoch 5/10
17743/17743 [==============================] - 195s 11ms/step - loss: 0.0720 - acc: 0.9758 - val_loss: 0.1070 - val_acc: 0.9722
Epoch 6/10
17743/17743 [==============================] - 196s 11ms/step - loss: 0.0655 - acc: 0.9775 - val_loss: 0.1094 - val_acc: 0.9700
Epoch 7/10
17743/17743 [==============================] - 192s 11ms/step - loss: 0.0577 - acc: 0.9804 - val_loss: 0.15